# Solving Sudoku Puzzles

## Getting Started
This jupyter notebook presents an agent that can solve sudoku puzzles. A 9x9 grid is given with some fixed values. To solve the puzzle, the objective is to fill the empty cells of the grid such that the numbers 1 to 9 appear exactly once in each row, column, and 3x3 block of the grid. 

Below is a sample puzzle along with its solution. 

<img src="images/sudoku.png" style="width: 50%;"/>

### Choice of Algorithm
The choice of different algorithms to solve sudoku puzzles is explained the read me file and implemented in cells below, all approaches consider *data structures*, as tesult of certain algorithms will work much faster with certain data structures.


## Sample Sudoku Puzzles
To get started, the cell below will load in some sample sudoku puzzles for you so you can see the format. There are sudokus provided of multiple difficulties (easier sudokus typically start with more digits provided). The cell below only loads the easiest, but there is another test cell lower in the notebook which will run your code against all of the provided puzzles.

Each sudoku is a 9x9 NumPy array of integers, where zero represents an empty square. Each difficulty comes with 15 sudokus, so when you load the file, it is stored in a 15x9x9 array.

In [1]:
import numpy as np

# Load sudokus
sudoku = np.load("data/very_easy_puzzle.npy")
print("very_easy_puzzle.npy has been loaded into the variable sudoku")
print(f"sudoku.shape: {sudoku.shape}, sudoku[0].shape: {sudoku[0].shape}, sudoku.dtype: {sudoku.dtype}")

# Load solutions for demonstration
solutions = np.load("data/very_easy_solution.npy")
print()

# Print the first 9x9 sudoku...
print("First sudoku:")
print(sudoku[0], "\n")

# ...and its solution
print("Solution of first sudoku:")
print(solutions[0])

very_easy_puzzle.npy has been loaded into the variable sudoku
sudoku.shape: (15, 9, 9), sudoku[0].shape: (9, 9), sudoku.dtype: int8

First sudoku:
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]] 

Solution of first sudoku:
[[1 7 4 3 8 2 9 5 6]
 [2 9 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 7 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 1 7 2 9 8 5 4 3]
 [8 4 3 7 1 5 2 6 9]]


## Part One

Backtracking method for the puzzle


In [2]:
import time
import numpy as np
import copy


################################################
###              revise
################################################
def revise(sudoku): # Revise if sudoku has no repeated values
    
    revised_sudoku = copy.deepcopy(sudoku)
    
    # Checking for row
    for row in range(9):
        
        row_array = revised_sudoku[row] # 1D empty array 
        variables, counts = np.unique(row_array, return_counts=True) # Counting number of time the variable is repeated
        
        if checking_reps(variables, counts, row_array):
            return True
    
    # Checking for column / Transpose method to avoid nested for loops
    for column in range(9):
        
        column_array = revised_sudoku.transpose()[column] # 1D empty array 
        variables, counts = np.unique(column_array, return_counts=True) # Counting number of time the variable is repeated
        
        if checking_reps(variables, counts, column_array):
            return True
        
    # Checking for blocks
    i = 0
    j = 0
    
    while j < 9:
        
        if i == 9:
            i = 0
            revised_sudoku = np.delete(revised_sudoku, (0,1,2), axis=1) # Deleting columns from 0 to 2, axis = 1 for column and 0 for row
       
        block_array = np.zeros(0) # 1D empty array    

        for row in range(3):
            for column in range(3):

                block_array = np.append(block_array, revised_sudoku[row + i, column]) # Adding the numbers / variables inside the block
        
        variables, counts = np.unique(block_array, return_counts=True) # Counting number of time the variable is repeated

        if checking_reps(variables, counts, block_array):
            return True
    
        i += 3      
        j +=1
           
    return False


################################################
###              checking_reps
################################################
def checking_reps(variables, counts, array): # Complement function to check repeated values
    
    if (array == np.zeros(9)).all(): # All values in the array are zero
        return False
    
    if np.amin(variables) == 0:
        counts = np.delete(counts,0) # Deleting index 0, which is the spot for empty spaces (0)
        
    if np.amax(counts) > 1:
        #print("STOP")
        return True # True indicates at leat one value in domain is repetaed
    
    return False # False indicates no values in domian are repeated


################################################
###              replacing_zeros
################################################
def replacing_zeros(sudoku): # Replace all 0s with all nine possible values {1,2,3,4,5,6,7,8,9}
    
    full_values_sudoku = copy.deepcopy(sudoku)
    full_values_sudoku = full_values_sudoku.tolist() # Converting array to a list in order to add an array inside the cells
    
    for row in range(9):
        for column in range(9):
            cell = full_values_sudoku[row][column]
            if cell == 0:
                full_values_sudoku[row][column] = np.arange(1,10) #Adding the array in a cell
    full_values_sudoku = np.array(full_values_sudoku, dtype=object)    
    
    return full_values_sudoku


################################################
###              goal
################################################
def goal(state): # Function to know if the sudoku was fully filled

    for row in state:
        for cell in row:
            if isinstance(cell, np.ndarray): # If cell is still an array  we are NOT done
                return False
    return True


################################################
###              propagate_r_c_b
################################################
def propagate_r_c_b(state, new_constrains = False): #Propogate values to row, columns and blocks depending on fixed_values and constrains
    
    # Propagating the fixed values (constrains) to rows

    for r in range(9): # To cover each row
        
        constrains = np.array([]) # Empty array to save constrains (propagated and fixed values / already set in the sudoku)
        row = state[r] # Getting row with all their elements
        
        for fixed_values in row: # To save fixed_values set in initial sudoku
            
            # Adding the fixed values if they are not an array which contains the domain {1,2,3,4,5,6,7,8,9}
            if not isinstance(fixed_values, np.ndarray): 
                constrains = np.append(constrains, fixed_values)
    
        for c in range(9): # To cover element by element in sudoku
            
            # Only enter if the cell (statet[r][c]) is an array with all possible values
            if isinstance(state[r][c], np.ndarray):  
                
                state[r][c] = np.setdiff1d(state[r][c], constrains) # Deleting all previous constrains in the specific cell
                                
                # Once all the constrains were applied, one reaming value is left in the array per cell
                if len(state[r][c]) == 1: 
                    val=state[r][c][0] # Saving only the last value in form of integer
                    state[r][c]=val # Replacing the array in the cell with the integer and final value
                    
                    constrains = np.append(constrains, val) # The value is added as a constrain to be used along row length
                    constrains.sort()
                    
                    new_constrains = True # True means a new constrain was added successfully and ready to move on
                    
                # 
                elif len(state[r][c]) == 0:
                    
                    return False, None           
   
    # Propagating the fixed values (constrains) to columns
    for c in range(9):
        
        constrains = np.array([]) # Empty array to save constrains (propagated and fixed values / already set in the sudoku)
        column = state.transpose()[c] # Getting columns with all their elements
        
        for fixed_values in column: # To save fixed_values set in initial sudoku
            
            # Adding the fixed values if they are not an array which contains the domain {1,2,3,4,5,6,7,8,9}
            if not isinstance(fixed_values, np.ndarray): 
                constrains = np.append(constrains, fixed_values)
                
        for r in range(9): # To cover element by element in sudoku
            
            # Only enter if the cell (statet[r][c]) is an array with all possible values
            if isinstance(state[r][c], np.ndarray):  
                
                state[r][c] = np.setdiff1d(state[r][c], constrains) # Deleting all previous constrains in the specific cell
                                
                # Once all the constrains were applied, one reaming value is left in the array per cell
                if len(state[r][c]) == 1: 
                    val=state[r][c][0] # Saving only the last value in form of integer
                    state[r][c]=val # Replacing the array in the cell with the integer and final value
                    
                    constrains = np.append(constrains, val) # The value is added as a constrain to be used along row length
                    constrains.sort()
                    
                    new_constrains = True # True means a new constrain was added successfully and ready to move on
                    
                # 
                elif len(state[r][c]) == 0:
                    
                    return False, None

    # Propagating the fixed values (constrains) to blocks
    
    for index_row in range(3): #Covering block by block with their respective indices
        for index_column in range(3):
            
            constrains = np.array([]) # Empty array to save constrains (propagated and fixed values / already set in the sudoku)
            
            row_start = index_row * 3 # Posiible values are {0, 3, 6}
            col_start = index_column * 3 # Posiible values are {0, 3, 6}

            for r in range(row_start, row_start + 3): # To cover horizontal blocks
                for c in range(col_start, col_start + 3): # To cover vertical blocks

                    fixed_values_cell = state[r][c]
                    
                    # Adding the fixed values if they are not an array which contains the domain {1,2,3,4,5,6,7,8,9}
                    if not isinstance(fixed_values_cell, np.ndarray): 
                        constrains = np.append(constrains, fixed_values_cell)
                    
            for r in range(row_start, row_start + 3): # To cover horizontal blocks
                for c in range(col_start, col_start + 3): # To cover vertical blocks
                    
                    # Only enter if the cell (statet[r][c]) is an array with all possible values
                    if isinstance(state[r][c], np.ndarray):  
                
                        state[r][c] = np.setdiff1d(state[r][c], constrains) # Deleting all previous constrains in the specific cell
                        
                        # Once all the constrains were applied, one reaming value is left in the array per cell
                        if len(state[r][c]) == 1: 
                            val=state[r][c][0] # Saving only the last value in form of integer
                            state[r][c]=val # Replacing the array in the cell with the integer and final value

                            constrains = np.append(constrains, val) # The value is added as a constrain to be used along row length
                            constrains.sort()

                            new_constrains = True # True means a new constrain was added successfully and ready to move on

                        # 
                        elif len(state[r][c]) == 0:

                            return False, None
            
    return True, new_constrains


################################################
###              propagation
################################################
def propagation(full_values_sudoku): #Complement funtion to create a loop and get a solution
    
    while True:
        solvable, new_constrains = propagate_r_c_b(full_values_sudoku)
        if solvable == False: # No solution
            return False
        if new_constrains == False: # Iterative process and adding new constrains
            return True

################################################
###              next_move
################################################
def next_move(sudoku, row, column, number): # Verifying the next move
    #print(f"row:{row},column:{column},number={number}")
    
    # Checking if the number is in the row
    for r in range(9):
        if sudoku[row, r] == number:
            return False
    
    # Checking if the number is in the column
    for c in range(9):
        if sudoku[c, column] == number:
            return False
    
    row_start = (row // 3) * 3 # 10 // 3 = 3, 5 // 3 = 1, 1 // 3 = 0
    col_start = (column // 3) * 3

    for r in range(row_start, row_start + 3):
        for c in range(col_start, col_start + 3):
            if sudoku[r, c] == number:
                return False 
    return True


################################################
###              counting_zeros
################################################
def counting_zeros(a): # Function to count 0s per column and row
    
    array_number_of_zeros_row = np.empty(9, dtype=int) # 1D array to save number of zeros in a row
    array_number_of_zeros_column = np.empty(9, dtype=int) # 1D array to save number of zeros in a column
    

    for i in range(9):
        
        array_number_of_zeros_row[i] = np.count_nonzero(a[i] == 0) # Counting zeros per row
        array_number_of_zeros_column[i] = np.count_nonzero(a.transpose()[i] == 0) # Counting zeros per column
        
    return array_number_of_zeros_row, array_number_of_zeros_column


################################################
###              find_next_section
################################################
def find_next_section(a, array_number_of_zeros_row, array_number_of_zeros_column): # Function to find rows and columns with fewer 0s
    
    for i in range(9):
        if array_number_of_zeros_row[i] <= array_number_of_zeros_column[i]:
    
            return next_section(a, array_number_of_zeros_row, Row = True)
    
        else:
            
            return next_section(a, array_number_of_zeros_column, Column = True)
            
    return None, None


################################################
###              next_section
################################################
def next_section(a, array_number_of_zeros, Row = False, Column = False): #Complement function to find rows and columns with fewer 0s
    
    # Row (Minumun values and index)
    min_value = np.amin(array_number_of_zeros) # Finding the minimum value in the row
    index_min_value = np.argmin(array_number_of_zeros) # Finding the row (index) with lowest number of zeros
    
    #min_column = np.argmin(array_number_of_zeros_column) # Finding the column with lowest number of zeros
    
    while min_value == 0:
    
        array_number_of_zeros[index_min_value] = 10 # Change the value to 10 we they reach 0
        min_value = np.amin(array_number_of_zeros) # Finding the minimum value in the row once we complete the previous row
        index_min_value = np.argmin(array_number_of_zeros) # Finding the row (index) with lowest number of zeros
        
        if (array_number_of_zeros == np.full((9),10)).all():
            return None, None
       
    if Row:
        # Findig the column index for the given min_row
        for i in range(9):
            if a[index_min_value][i] == 0:

                new_row = index_min_value # Index for row with lowest number of zero
                new_column = i # Corresponding column with less 0

                return new_row, new_column
            
    if Column:
        # Findig the column index for the given min_row
        for i in range(9):
            if a[i][index_min_value] == 0:

                new_column = index_min_value #Index for column with lowest number of zero
                new_row = i # Corresponding row with less 0 
                
                return new_row, new_column

    return None, None


################################################
###              backtracking_backup
################################################
def backtracking_backup(sudoku): #Second backtracking process
    
    array_row, array_column = counting_zeros(sudoku)

    row, column = find_next_section(sudoku, array_row, array_column)
    
    # Goal Achieved
    if row == column == None:
        return True
    
    #print(f"row:{row}, col: {column}")
    
    # Backtracking
    for num in range (1, 10):
        if next_move(sudoku, row, column, num):
            sudoku[row, column] = num
            
            if backtracking_backup(sudoku):
                return True
            
        sudoku[row, column] = 0
    return False


################################################
###              backtracking
################################################
def backtracking(full_values_sudoku): #Main backtracking process

    solvable = propagation(full_values_sudoku)

    if not solvable:
        return False

    if goal(full_values_sudoku):
        return full_values_sudoku

    for row in range(9):
        for column in range(9):
            
            cell = full_values_sudoku[row][column]
            if isinstance(cell, np.ndarray):
                for values in cell:
                    new_state = copy.deepcopy(full_values_sudoku)
                    new_state[row][column] = values
                    final_state = backtracking(new_state)
                    if final_state is not False:
                        return final_state
                return False
    return False


################################################
###              sudoku_solver
################################################
def sudoku_solver(sudoku): #Main function
    
    if revise(sudoku): # Sudoku with repeated values 
        #print(np.full((9, 9), -1))
        return np.full((9, 9), -1)
      
    full_values_sudoku = replacing_zeros(sudoku)
    feasible_solution = backtracking(full_values_sudoku)
    
    if feasible_solution is not False:
        
        if revise(feasible_solution): # Sudoku with repeated values 
            
            solution_2 = backtracking_backup(sudoku)
            
            if solution_2:
        
                if revise(sudoku):
                    return np.full((9, 9), -1)
                    #print(np.full((9, 9), -1))
                
                return sudoku
                #print(sudoku)
            
            #print(np.full((9, 9), -1))
            return np.full((9, 9), -1)
        
        #print(feasible_solution)
        return feasible_solution
    
    return np.full((9, 9), -1)
    #print(np.full((9, 9), -1))
    

sudoku = np.array([[0, 2, 0, 0, 0, 6, 9, 0, 0],
                   [0, 0, 0, 0, 5, 0, 0, 2, 0],
                   [6, 0, 0, 3, 0, 0, 0, 0, 0],
                   [9, 4, 0, 0, 0, 7, 0, 0, 0],
                   [0, 0, 0, 4, 0, 0, 7, 0, 0],
                   [0, 3, 0, 2, 0, 0, 0, 8, 0],
                   [0, 0, 9, 0, 4, 0, 0, 0, 0],
                   [3, 0, 0, 9, 0, 2, 0, 1, 7],
                   [0, 0, 8, 0, 0, 0, 0, 0, 2]])

#start = time.time()


sudoku_solver(sudoku)
#end = time.time()
#print(f"The execution time was:  {end-start} secs")

assert((sudoku_solver(sudoku) == np.array([[4, 2, 5, 8, 1, 6, 9, 7, 3],
       [8, 9, 3, 7, 5, 4, 6, 2, 1],
       [6, 1, 7, 3, 2, 9, 5, 4, 8],
       [9, 4, 1, 6, 8, 7, 2, 3, 5],
       [5, 8, 2, 4, 3, 1, 7, 6, 9],
       [7, 3, 6, 2, 9, 5, 1, 8, 4],
       [2, 7, 9, 1, 4, 8, 3, 5, 6],
       [3, 5, 4, 9, 6, 2, 8, 1, 7],
       [1, 6, 8, 5, 7, 3, 4, 9, 2]])).all())


### Testing
There are four difficulties of sudoku provided: very easy, easy, medium, and hard. There are 15 sample sudokus in each category, with solutions as well. Difficulty was determined using reference solvers, but the code may vary.

*All categories that are easy and above will contain* ***invalid initial states***, that is, sudoku puzzles with no solution. In this case, your function should return a 9x9 NumPy array whose values are all equal to -1.

The agent is firstly tested with very easy puzzles. Then it is tested on additional *hidden* sudokus from each difficulty in turn, easy and up – many more than 15 of each. 

All puzzles must take under 20 seconds each on the test machine to count as successful – if it takes longer, it will timeout.

***The hard sudokus are labelled as hard for a reason.*** 

The best way to improve the performance of the code is through a detailed understanding and smart choice of AI algorithms and data structures. 

#### Test Cell
The following code will run your solution over the provided sudoku puzzles. To enable it, set the constant `SKIP_TESTS` to `False`.

**IMPORTANT**: you must set `SKIP_TESTS` back to `True` before submitting this file!

In [3]:
SKIP_TESTS = True

def tests():
    import time
    difficulties = ['very_easy', 'easy', 'medium', 'hard']

    for difficulty in difficulties:
        print(f"Testing {difficulty} sudokus")
        
        sudokus = np.load(f"data/{difficulty}_puzzle.npy")
        solutions = np.load(f"data/{difficulty}_solution.npy")
        
        count = 0
        for i in range(len(sudokus)):
            sudoku = sudokus[i].copy()
            print(f"This is {difficulty} sudoku number", i)
            print(sudoku)
            
            start_time = time.process_time()
            your_solution = sudoku_solver(sudoku)
            end_time = time.process_time()
            
            print(f"This is your solution for {difficulty} sudoku number", i)
            print(your_solution)
            
            print("Is your solution correct?")
            if np.array_equal(your_solution, solutions[i]):
                print("Yes! Correct solution.")
                count += 1
            else:
                print("No, the correct solution is:")
                print(solutions[i])
            
            print("This sudoku took", end_time-start_time, "seconds to solve.\n")

        print(f"{count}/{len(sudokus)} {difficulty} sudokus correct")
        if count < len(sudokus):
            break
            
if not SKIP_TESTS:
    tests()

In [4]:
def submission_tests():
    import sys
    import pathlib

    fail = False;

    if not SKIP_TESTS:
        fail = True;
        print("You must set the SKIP_TESTS constant to True in the cell above.")

    p1 = pathlib.Path('./readme.txt')
    p2 = pathlib.Path('./readme.md')
    if not (p1.is_file() or p2.is_file()):
        fail = True;
        print("You must include a separate file called readme.txt or readme.md in your submission.")

    p3 = pathlib.Path('./sudoku.ipynb')
    if not p3.is_file():
        fail = True
        print("This notebook file must be named sudoku.ipynb")

    if "sudoku_solver" not in globals():
        fail = True;
        print("You must include a function called sudoku_solver which accepts a numpy array.")
    else: 
        sudoku = np.load("data/very_easy_puzzle.npy")[0]
        solution = np.load("data/very_easy_solution.npy")[0]

        if not np.array_equal(sudoku_solver(sudoku), solution):
            print("Warning:")
            print("Your sudoku_solver function does not correctly solve the first sudoku.")
            print()
            print("Your assignment is unlikely to get any marks from the autograder. While we will")
            print("try to check it manually to assign some partial credit, we encourage you to ask")
            print("for help on the forum or directly to a tutor.")
            print()
            print("Please use the readme file to explain your code anyway.")

    if fail:
        print()
        sys.stderr.write("Your submission is not ready! Please read and follow the instructions above.")
    else:
        print("All checks passed. When you are ready to submit, upload the notebook and readme file to the")
        print("assignment page, without changing any filenames.")
        print()
        print("If you need to submit multiple files, you can archive them in a .zip file. (No other format.)")
        
submission_tests()

All checks passed. When you are ready to submit, upload the notebook and readme file to the
assignment page, without changing any filenames.

If you need to submit multiple files, you can archive them in a .zip file. (No other format.)


In [5]:
# This is a TEST CELL. Do not delete or change.